In [1]:
# Insérez ici le numéro de votre équipe
VOTRE_NUMERO_EQUIPE = 0
VOTRE_STATION = 1  # votre poste de travail au laboratoire [1, 2, 3 ou 4]
VOTRE_ROBOT_REEL = 2  # indiqué derrière et manette Playsation (pas 02, juste 2 par exemple)
# modifier cette variable selon que vous contrôler la simulation ou le robot réel
SIM = False

## Import des paquets Python

In [2]:
# Import des paquets requis et configuration du ROS Master
import sys; sys.path.append('/opt/ros/noetic/lib/python3/dist-packages')
sys.path.append('/usr/lib/python3/dist-packages')
sys.path.append('/home/admin_mec/mobile_manip_ws/devel/lib/python3/dist-packages/')
from os import environ
if SIM:
    #environ['ROS_MASTER_URI'] = "http://localhost:1137{}/".format(VOTRE_NUMERO_EQUIPE)
    environ['ROS_MASTER_URI'] = "http://localhost:11311/" #Ordi personnel
else:
    environ['ROS_MASTER_URI'] = "http://cpr-ets05-0{}:11311/".format(VOTRE_ROBOT_REEL)
    environ['ROS_IP'] = "192.168.0.8{}".format(VOTRE_STATION)
import rospy
import numpy as np
from jackal_msgs.msg import Drive
from geometry_msgs.msg import Pose
from geometry_msgs.msg import Twist as GT
from sensor_msgs.msg import CompressedImage
from apriltag_ros.msg import AprilTagDetectionArray
from mobile_manip.srv import ReachName, GetValues, ReachValues

In [3]:
# Créer et démarrer un nouveau noeud
rospy.init_node('mobile_manip_controller', anonymous=True)

## Création des 'callbacks'
Chaque fonction est associée à un sujet ROS et enregistre en continue sont contenu dans une variable globale.

In [4]:
# AprilTag subscriber callback
tag_array = AprilTagDetectionArray()
def tag_callback(msg):
    global tag_array
    tag_array  = msg

In [5]:
# Camera subscriber callback
cam_msg = CompressedImage()
def cam_callback(msg):
    global cam_msg
    cam_msg = msg

In [6]:
# ROS subscribers et publishers
cmd_drive_pub = rospy.Publisher('/mobile_manip/dingo_velocity_controller/cmd_drive', Drive, queue_size=1)
if SIM:
    cmd_vel_pub = rospy.Publisher('/mobile_manip/cmd_vel', GT, queue_size=1)
else:
    cmd_vel_pub = rospy.Publisher('/mobile_manip/base/cmd_vel', GT, queue_size=1)
cam_sub = rospy.Subscriber('/mobile_manip/d435i/color/image_raw/compressed', CompressedImage, cam_callback)
tag_sub = rospy.Subscriber('/tag_detections', AprilTagDetectionArray, tag_callback)

## Contrôleur du mouvement

In [7]:
interwheel_distance = 0.3765
left_wheel_radius = 0.098
right_wheel_radius = 0.098
def move_robot(linear, angular):
    vel_left  = (linear - angular * interwheel_distance / 2.0) / left_wheel_radius
    vel_right = (linear + angular * interwheel_distance / 2.0) / right_wheel_radius

    if SIM:
        # Envoi des commandes au roues par topic ROS
        cmd_drive_msg = Drive()
        cmd_drive_msg.drivers[0] = vel_left
        cmd_drive_msg.drivers[1] = vel_right
        cmd_drive_pub.publish(cmd_drive_msg)

    else:
        # Envoi des commandes au controlleur du robot
        cmd_vel_msg = GT()
        cmd_vel_msg.linear.x = linear
        cmd_vel_msg.angular.z = angular
        cmd_vel_pub.publish(cmd_vel_msg)


## Interface manuelle du Dingo
Permet le contrôle du robot avec des flèches et un retour visuel de la caméra frontale.

In [8]:
import ipywidgets as widgets
from ipywidgets import GridspecLayout
import threading
import time

btn_up = widgets.Button(icon='arrow-up')
btn_left = widgets.Button(icon='arrow-left')
btn_down = widgets.Button(icon='arrow-down')
btn_right = widgets.Button(icon='arrow-right')
btn_stop = widgets.Button(description='Stop')
image = widgets.Image(
        value=cam_msg.data,
        format="png",
        height="40%",
        width="60%")

def update_plot():
    while True:
        image.value=cam_msg.data
        time.sleep(0.1)

def on_btn_up_clicked(b):
    move_robot(0.5,0)
        
def on_btn_left_clicked(b):
    move_robot(0,1.0)
        
def on_btn_down_clicked(b):
    move_robot(-0.5,0)
        
def on_btn_right_clicked(b):
    move_robot(0,-1.0)
    
def on_btn_stop_clicked(b):
    move_robot(0,0)

btn_up.on_click(on_btn_up_clicked)
btn_left.on_click(on_btn_left_clicked)
btn_down.on_click(on_btn_down_clicked)
btn_right.on_click(on_btn_right_clicked)
btn_stop.on_click(on_btn_stop_clicked)


grid = GridspecLayout(4, 3,height='600px')

grid[0,:] = image
grid[1,1] = btn_up
grid[2,0] = btn_left
grid[2,1] = btn_stop
grid[2,2] = btn_right
grid[3,1] = btn_down
thread = threading.Thread(target=update_plot)
thread.start()
grid

GridspecLayout(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xf…

## Fonctions de contrôle du Gen3Lite

In [9]:
# Pour lancer une position prédéfinie
rospy.wait_for_service('/mobile_manip/reach_name')
reach_recorded_pose = rospy.ServiceProxy('/mobile_manip/reach_name', ReachName)
# pre-recorded pose are home, retract and vertical
res1 = reach_recorded_pose("vertical")
print(res1)

res: False


In [10]:
# Pour obtenir les coordonnées actuelles de la pince
rospy.wait_for_service('/mobile_manip/get_cartesian')
get_pose = rospy.ServiceProxy('/mobile_manip/get_cartesian', GetValues)
res2 = get_pose()
print(res2)

val: [0.0571092888712883, -0.009969281032681465, 1.0032371282577515, 0.012723485939204693, 0.010381439700722694, 89.97567749023438]


## Message de détection des marqueurs

In [11]:
from scipy.spatial.transform import Rotation as R
if SIM:
    T_cam2base = np.array([[0.16], [0.04], [0.463]])
else:
    T_cam2base = np.array([[0.0],[0.05], [-0.137]])
if not tag_array.detections:
        print("Aucun tag en vue")
else:
    for tag in tag_array.detections:
        print("Numero du marqueur : {}".format(tag.id[0]))
        print("Position du marqueur :")
        print(tag.pose.pose.pose.position)
        print("Quaternion du marqueur :")
        print(tag.pose.pose.pose.orientation)
        print("Eulers du marqueur :")
        rot = R.from_quat([tag.pose.pose.pose.orientation.x, tag.pose.pose.pose.orientation.y, tag.pose.pose.pose.orientation.z, tag.pose.pose.pose.orientation.w])
        print(rot.as_euler('zyx', degrees=True))
        if(tag.id[0]==8):
            grab_pose = tag.pose.pose.pose.position
            grab_orientation = tag.pose.pose.pose.orientation
        if(tag.id[0]==3):
            drop_pose = tag.pose.pose.pose.position
            drop_orientation = tag.pose.pose.pose.orientation

Numero du marqueur : 8
Position du marqueur :
x: -0.14075994042215131
y: -0.25706110681689914
z: 0.6720367155871447
Quaternion du marqueur :
x: 0.7194079594241873
y: -0.6888576189037131
z: 0.043041194697330386
w: 0.07794115956579006
Eulers du marqueur :
[ 87.28923356  -2.60512632 170.11804378]


## Tourne et touche le tag

In [12]:
from scipy.spatial.transform import Rotation as R
dist = 1.5
if SIM:
    T_cam2base = np.array([[0.16], [0.04], [0.463]])
else:
    T_cam2base = np.array([[0.0], [0.05], [-0.137]])

while dist > 0.95:
    # Tourne pour voir le tag
    while not tag_array.detections:
        move_robot(0.0, 0.5)
    tag_pose = tag_array.detections[0].pose.pose.pose.position
    tag_orientation = tag_array.detections[0].pose.pose.pose.orientation
    # Convertit le tag dans le référentiel du bras
    img_to_cam = R.from_euler('zyx',[-90,90,0],degrees=True)
    tag_in_base_arm = np.matmul(img_to_cam.as_matrix(),np.array([[tag_pose.x],[tag_pose.y],[tag_pose.z]]))+T_cam2base
    dist = np.sqrt((tag_in_base_arm[0])**2+(tag_in_base_arm[1])**2)
    move_robot(0.1,0.0)

print(tag_in_base_arm)
move_robot(0, 0)

[[0.67182328]
 [0.19070112]
 [0.1200012 ]]


In [13]:
# Touche le Tag
# rospy.wait_for_service('/mobile_manip/reach_cartesian')
# reach_pose = rospy.ServiceProxy('/mobile_manip/reach_cartesian', ReachValues)
# effector_goal = [tag_in_base_arm[0], tag_in_base_arm[1], tag_in_base_arm[2], 0, 90, 0, 0.09]
# print(effector_goal)
# res3 = reach_pose(effector_goal)
# print(res3)

## Ramasse la bouteille

### Ouvre la pince et revient à la vertical

In [14]:
rospy.wait_for_service('/mobile_manip/reach_gripper')
reach_grip = rospy.ServiceProxy('/mobile_manip/reach_gripper', ReachValues)
effector_open = [0.0]
res3 = reach_grip(effector_open)
rospy.wait_for_service('/mobile_manip/reach_name')
reach_recorded_pose = rospy.ServiceProxy('/mobile_manip/reach_name', ReachName)
res = reach_recorded_pose("vertical")
print(res3)

res: True


In [15]:
# ATTENTION, il faut approcher la bouteille pour ne pas la faire tomber.
# Connaissant sa position p/r au tag
# On arriver par-dessus ou en face avant de s'y rendre pour la préhension.
hauteur_bouteille = 0.20

In [16]:
img_to_cam = R.from_euler('zyx',[-90,90,0],degrees=True)

tag_to_img = R.from_quat([grab_orientation.x,grab_orientation.y,grab_orientation.z,grab_orientation.w])
img_to_cam = R.from_euler('zyx',[-90,90,0],degrees=True)

# Prennez vos mesures pour vous assurez des bons chiffres
bouteille_fromtag = np.array([[0.050], [-0.680], [0.480]])

bouteille_frombase = np.matmul(img_to_cam.as_matrix(),np.matmul(tag_to_img.as_matrix(),bouteille_fromtag))+tag_in_base_arm
print(bouteille_frombase)

[[ 0.17196477]
 [-0.4683811 ]
 [ 0.22513013]]


In [17]:
rospy.wait_for_service('/mobile_manip/reach_cartesian')
reach_pose = rospy.ServiceProxy('/mobile_manip/reach_cartesian', ReachValues)

# Ici on ajoute la hauteur de la bouteille + un buff pour arriver par dessus
buff = 0.05
effector_goal = [bouteille_frombase[0], bouteille_frombase[1], bouteille_frombase[2]+hauteur_bouteille+buff, 90, 0, 0, 0.01]
print(effector_goal)
res3 = reach_pose(effector_goal)
print(res3)

[array([0.17196477]), array([-0.4683811]), array([0.47513013]), 90, 0, 0, 0.01]
res: False


In [18]:
# On descend !
rospy.wait_for_service('/mobile_manip/get_cartesian')
get_pose = rospy.ServiceProxy('/mobile_manip/get_cartesian', GetValues)
res2 = get_pose()
currentpose = list(res2.val)
currentpose[2] = currentpose[2]-buff-hauteur_bouteille/2  # 10 prend la bouteille à son centre
currentpose.append(0.25)
print(currentpose)
res3 = reach_pose(currentpose)
print(res3)

[0.05726919323205948, -0.009957121685147285, 0.8532176971435547, 0.032194193452596664, 0.012521139346063137, 89.98355865478516, 0.25]
res: False


### Ferme la pince

In [19]:
rospy.wait_for_service('/mobile_manip/reach_gripper')
reach_grip = rospy.ServiceProxy('/mobile_manip/reach_gripper', ReachValues)
effector_open = [0.8]
res3 = reach_grip(effector_open)
print(res3)

res: True


In [20]:
# On lève !
rospy.wait_for_service('/mobile_manip/get_cartesian')
get_pose = rospy.ServiceProxy('/mobile_manip/get_cartesian', GetValues)
res2 = get_pose()
currentpose = list(res2.val)
currentpose[2] = currentpose[2]+0.10  # soulève de 10 cm pour s'éloigner de la table
currentpose.append(0.25)
print(currentpose)
res3 = reach_pose(currentpose)
print(res3)

[0.057277191430330276, -0.009954185225069523, 1.103216862678528, 0.03304852545261383, 0.012948252260684967, 89.98432159423828, 0.25]
res: False


## Déposer l'objet

## Retour au HOME